In [33]:
#Dependencies
import pandas as pd
import numpy as np
from citipy import citipy
import time
from datetime import datetime
import requests
from config2 import weather_api_key
import json
from pprint import pprint


In [34]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [36]:
#create a set of random latitude and longitude cominations
lats= np.random.uniform(low= -90.000, high=90.000, size= 2000)
longs= np.random.uniform(low= -180.000, high= 180.000, size= 2000)
lats_longs = zip(lats, longs)

coordinates = list(lats_longs)


In [43]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)


In [38]:
# create an empty list to hold the weather data
city_data= []

#print the beginning of the logging
print("Beginning Data Retrieval     ")
print("-----------------------------")

#create counters
record_count= 1
set_count= 1

#Loop through all the cities in our list
for i, city in enumerate(cities):

    #group in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    #create endpoint URL with each city
    city_url = url + "&q=" + city.replace(" ", "+")

    #Log the URL, record, and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} | {city}")

    #add 1 to the record count
    record_count += 1

#run an API request for each of the cities
    try:
        #parse the JSON and retrieve the data
        city_weather = requests.get(city_url).json()

        #parse out the needed data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humdity = city_weather["main"]["humidity"]
        city_clouds= city_weather["clouds"]["all"]
        city_wind= city_weather["wind"]["speed"]
        city_country= city_weather["sys"]["country"]
        weather_description= city_weather["weather"][0]["description"]

        #convert the date to ISO standard
        city_date= datetime.utcfromtimestamp(city_weather["dt"]).strftime("%Y-%m-%d %H:%M:%S")

        #append the city information into the city_data list.
        city_data.append({"City":city.title(),
                        "Lat": city_lat,
                        "Lng": city_lng,
                        "Max Temp": city_max_temp,
                        "Humidity": city_humdity,
                        "Cloudiness": city_clouds,
                        "Wind Speed": city_wind,
                        "Country": city_country,
                        "Description": weather_description})

#if an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

#indicate that the data loading is complete
print("---------------------------------")
print("Data Retrival Complete           ")
print("---------------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | dingle
Processing Record 2 of Set 1 | labytnangi
Processing Record 3 of Set 1 | kodiak
Processing Record 4 of Set 1 | kapaa
Processing Record 5 of Set 1 | saskylakh
Processing Record 6 of Set 1 | arrecife
Processing Record 7 of Set 1 | ushuaia
Processing Record 8 of Set 1 | bluff
Processing Record 9 of Set 1 | yekepa
Processing Record 10 of Set 1 | alofi
Processing Record 11 of Set 1 | tilichiki
Processing Record 12 of Set 1 | thompson
Processing Record 13 of Set 1 | bredasdorp
Processing Record 14 of Set 1 | victoria
Processing Record 15 of Set 1 | rikitea
Processing Record 16 of Set 1 | egvekinot
Processing Record 17 of Set 1 | whitianga
Processing Record 18 of Set 1 | iralaya
Processing Record 19 of Set 1 | saint-philippe
Processing Record 20 of Set 1 | fukue
Processing Record 21 of Set 1 | ural
Processing Record 22 of Set 1 | kenai
Processing Record 23 of Set 1 | albany
Processing Record 24 o

In [41]:
#convert city_data to DataFrame
city_data_df= pd.DataFrame(city_data)

#Sort dataframe to desired column order
new_column_order= ["City", "Country","Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Description"]

city_data_df= city_data_df[new_column_order]

#Export city_data_df to csv
city_data_df.to_csv("weatherPy_Database.csv", index_label="City_ID")